# Deriving Insights from Record Store Database

# Introduction

On this project we will be working with a record store named Chinook. The database that we use includes tables for artists, albums, media tracks, invoices and customers. There are three main questions that we will answer on this project:
* Which genres are most popular in the USA?
* Which sales agent is doing better than others? Why?
* Which country should we expand our marketing efforts towards?

For reference, the following is the schema of Chinook database.

In [1]:
%%html
<img src="img/chinook-schema.png", width="400">

# Overview of Data

Let's first set up our environment to accomodate SQL Lite and import the Chinook database.

In [2]:
import sqlite3 as sql

In [3]:
%%capture
%reload_ext sql

In [4]:
%sql sqlite:///chinook.db

Next, let's preview the data and run a couple of queries on the tables.

In [5]:
%%sql
SELECT 
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
albums,table
sqlite_sequence,table
artists,table
customers,table
employees,table
genres,table
invoices,table
invoice_items,table
media_types,table
playlists,table


In [6]:
%%sql
SELECT *     
FROM albums
LIMIT 5      

 * sqlite:///chinook.db
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [7]:
%%sql
SELECT *     
FROM invoices
ORDER BY InvoiceDate
LIMIT 5      

 * sqlite:///chinook.db
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


# Top Genres in the USA

Chinook just signed a deal with a new record label and we need to select the first three albums that will be added to the store. The record label has four albums on their list. All four albums are by artists that don't have any tracks in the store currently. The following are the artist names and the genre of music they produce:

| Artist Name          | Genre   |
|-                     |-        |
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in promoting artists from the USA, and have given Chinook some money to advertise new albums in the USA only. Let's find out which genres are the most popular in the country.

## Analysis

To approach this question, we will do a left join between tracks, invoice_items, invoices, and genres tables:

In [8]:
%%sql
SELECT 
    g.name,                          -- # selects the name of the genre
    SUM(ii.Quantity) tracks_sold,    -- # summation of the number of tracks sold
    CAST(SUM(ii.Quantity) as float) / (SELECT                          -- # divided by the total number of tracks sold in USA, not segmented by genre
                                       CAST(SUM(ii.Quantity) as float) 
                                       FROM invoice_items ii
                                       LEFT JOIN invoices i
                                       ON ii.InvoiceId = i.InvoiceId
                                       WHERE i.BillingCountry = 'USA') 
                                       percentage_sold

FROM tracks t 
LEFT JOIN invoice_items ii 
    ON t.TrackId = ii.TrackId
LEFT JOIN invoices i 
    ON ii.InvoiceId = i.InvoiceId
LEFT JOIN genres g
    ON t.GenreId = g.GenreId
WHERE i.BillingCountry = 'USA'    -- # filters the query to only show data for USA
GROUP BY g.name                   -- # segments the query by genre 
ORDER BY SUM(ii.Quantity) DESC    -- # sorts by the number of tracks sold

 * sqlite:///chinook.db
Done.


Name,tracks_sold,percentage_sold
Rock,157,0.31781376518218624
Latin,91,0.18421052631578946
Metal,64,0.12955465587044535
Alternative & Punk,50,0.10121457489878542
Jazz,22,0.044534412955465584
Blues,15,0.030364372469635626
TV Shows,14,0.02834008097165992
R&B/Soul,12,0.024291497975708502
Classical,8,0.016194331983805668
Comedy,8,0.016194331983805668


Based on the sales across different genres in the USA, we should prioritize the purchase of new albums by the following artists:
* Red Tone (Punk): Punk genre accounted for 50 track sales (10% of USA track sales)
* Slim Jim Bites (Blues): Blues genre accounted for 50 track sales (3% of USA track sales) 
* Meteor and the Girls (Pop): Pop genre accounted for 5 track sales (1% of USA track sales) 

However, these three genres only make up for 14.17% of the total sales for USA market. We should also be on the lookout for artists within the rock genre as they make up for 31.78% of the total sales. 

# Analyzing Employee Sales Performance

Each customer for Chinook Store gets assigned to a sales support agent when they first make a purchase. Management is interested in knowing which support agent is driving the most and least amount of sales, and what factors might be contributing to the sales agent's performance.

## Analysis

To approach this question, we will do a left join between employees, customers, and invoices tables to see how everyone's performing based on the sales number:

In [9]:
%%sql
SELECT 
    e.FirstName|| " "||e.LastName employee -- # combines the first and last name column as employee full name
    ,SUM(i.Total) Sales                    -- # summation of the sales number
FROM employees e
LEFT JOIN customers c
ON e.EmployeeId = c.SupportRepId
LEFT JOIN invoices i
ON i.CustomerId = c.CustomerId
GROUP BY e.FirstName|| " "||e.LastName
ORDER BY Sales DESC                     -- # sort by sales (highest to lowest)

 * sqlite:///chinook.db
Done.


employee,Sales
Jane Peacock,833.0400000000013
Margaret Park,775.4000000000011
Steve Johnson,720.160000000001
Andrew Adams,None
Laura Callahan,None
Michael Mitchell,None
Nancy Edwards,None
Robert King,None


Based on this result, Jane Peacock is the top performer and she generated about 15% more sales compared to Steve Johnson, the bottom performer. Next, let's see if there's a variable on the employee table that might contribute to their performance.

In [10]:
%%sql
SELECT 
   *
FROM employees e
WHERE e.LastName = "Johnson" 
    OR e.LastName = "Park"
    OR e.LastName = "Peacock" 

 * sqlite:///chinook.db
Done.


EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


It appears that Jane was hired over a year earlier than Steve. Since Jane has more experience, it is likely easier for her to communicate with the customers and led her to having better performance compared to her peers. 

Note: the invoice database only includes data from 2009 onwards, so it excludes any sales from customers on Jane's first year.

# Sales Data by Country

Chinook is looking to expand its marketing reach to other countries. We need to decide which countries to focus on since the store only have enough budget to expand towards three different countries. To come up with an answer we will gather the following data for each country:
* Total number of customers
* Total value of sales
* Average value of sales per customer
* Average order value

## Analysis

To approach this question, we will do a join between customers and invoices tables. To simplify our analysis, we will bucket countries that only have one customer to a group called "Other":

In [11]:
%%sql 
SELECT 
    c.country                                         
    ,COUNT(c.customerId) num_customer
    ,SUM(COALESCE(i.total,0)) sales
    ,SUM(i.total)/COUNT(c.customerId) avg_sales_per_cust
    ,SUM(i.total)/SUM(i.order_count) avg_order_value
FROM 
    (SELECT 
    c.CustomerId 
    ,CASE WHEN c2.num_customer = 1 then "Other"  -- # buckets countries with 1 customer to Other
     ELSE c.country END AS country
    ,CASE WHEN c2.num_customer = 1 then 1 ELSE 0 -- # enables sorting to have Other at the bottom or top of the result table
      END AS sort
    FROM customers c
    LEFT JOIN (SELECT                          -- # pulls number of customers by country
               COUNT(CustomerId) num_customer  
               ,country 
               FROM customers
               GROUP BY country) c2 
    ON c.country = c2.country
    GROUP BY c.CustomerId) c
LEFT JOIN (SELECT                               -- # pulls sales number and order count from invoices table           
           CustomerId
           ,SUM(COALESCE(Total,0)) total            -- # summation of sales number
           ,COUNT(distinct InvoiceID) order_count   -- # counts the number of orders
           FROM invoices
           GROUP BY customerid) i
ON i.CustomerId = c.CustomerId
GROUP BY c.country                               -- # segments result by country
ORDER BY c.sort, avg_sales_per_cust DESC                      -- # sort Other countries to the bottom, and countries with the highest sales at the top

 * sqlite:///chinook.db
Done.


country,num_customer,sales,avg_sales_per_cust,avg_order_value
Czech Republic,2,90.24000000000001,45.120000000000005,6.445714285714287
USA,13,523.0600000000001,40.23538461538462,5.747912087912089
Germany,4,156.48000000000002,39.120000000000005,5.588571428571429
France,5,195.10000000000002,39.02,5.574285714285715
Portugal,2,77.24000000000001,38.620000000000005,5.517142857142858
Brazil,5,190.10000000000002,38.02,5.431428571428572
Canada,8,303.96000000000004,37.995000000000005,5.427857142857143
India,2,75.26,37.63,5.78923076923077
United Kingdom,3,112.86000000000001,37.620000000000005,5.3742857142857146
Other,15,604.3,40.28666666666666,5.755238095238095


Based on the data, there may be opportunity in the following countries:
* Czech Republic
* Germany
* France

Czech Republic is the best country to spend additional marketing efforts towards because customers from this country have the highest average order value and average sales per customer. Therefore, each additional customer and/or order from Czech Republic will bring the highest amount of return on investment. Next on the list of countries with high average sales per customer are USA, Germany and France. Since we already have a strong presence in the USA, the next best countries to market towards are Germany and France.

It is also worth keeping in mind that the sample size from these countries is relatively low. Therefore, it is best to not invest too much on these countries right at the beginning because the sample size is not large enough to give us confidence. The best approach is to invest in smaller amounts in these countries, collect and analyze new customers to see if the trend still holds with the new customers.